In [ ]:
import os
import json
import utils
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def annotate(word, ents_list, output):
    for ents in ents_list:
        if word.idx >= ents['start'] and word.idx < ents['end']:
            if ents['count'] == 0:
                o = "{} {} {} {} {} {}".format(word.text, "B-" + ents['type'], word.pos_, word.lemma_, word.tag_, word.dep_)
                output.append(o)
                ents['count'] += 1
            else:
                o = "{} {} {} {} {} {}".format(word.text, "I-" + ents['type'], word.pos_, word.lemma_, word.tag_, word.dep_)
                output.append(o)
            return output
        
    o = "{} {} {} {} {} {}".format(word.text, "0", word.pos_, word.lemma_, word.tag_, word.dep_)
    output.append(o)
    return output

In [ ]:
def process_file(filename, out_file):
    ents_list = []
    output = []
    ## open file
    with open(filename) as f:
        if os.stat(filename).st_size != 0:
            d = json.load(f)
            doc = nlp(d["content"]) ## content as the original text
            hopper = d["cyberevent"]["hopper"] ## hopper array
            for h in hopper:
                events = h["events"] ## events array
                for E in events:
                    event_type = E["type"] ## example=> type: Attack
                    nugget = E["nugget"]
                    event_subtype = E["subtype"] ## example=> subtype: Databreach
                    if "argument" in E:
                        arguments = E["argument"] ## arguments array
                        for T in arguments:
                            t = T["type"] ## example=> type: Organization or Person or System
                            role_type = T["role"]["type"] ## example=> type: Victim or Attacker or
                            startOffset = T["startOffset"]
                            endOffset = T["endOffset"]
                            ents = {'type': role_type, 'start': startOffset, 'end':endOffset, 'count': 0}
                            ents_list.append(ents)

            for word in doc:
                output = annotate(word, ents_list, output)

    return output

In [ ]:
def process_dataset():
    directory = r'data/annotation/'
    directory_output = 'data/annotation_j/'
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            in_file = os.path.join(directory, filename)
            out_file = os.path.join(directory_output, filename.split(".")[0] + "_j.txt")
            print(in_file)
            output = process_file(in_file, out_file)
            with open(out_file, 'w') as filehandle:
                filehandle.writelines("%s\n" % line for line in output)
            print("Done.")

In [ ]:
process_dataset()